In [2]:
import pandas as pd
from io import BytesIO
from PIL import Image
import requests
import os

In [3]:
df = pd.read_parquet("hf://datasets/bghira/photo-concept-bucket/photo-concept-bucket.parquet")

c:\Users\farns\anaconda3\envs\pytorch\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Confirm that the data is loaded and column of url exists
df.head(1)

,id,class_label,type,slug,description,alt,created_at,title,location,tags,main_color,colors,width,height,aspect_ratio,url,cogvlm_caption,megapixels
271389,474,quantum mechanics,photo,black-and-white-car-vehicle-vintage,,Person Fixing Car,2015-05-16T15:42:51.000Z,Person Fixing Car,None,"black and white, car, classic, garage, job, me...","[89, 89, 89]","[#3a3a3a, #8c8c8c, #cacaca]",6000,4000,1.5,https://images.pexels.com/photos/474/black-and...,A person is working underneath a car in a gara...,24.0


In [ ]:
index = 0
# I want 200 images
output_dir = '../images/training_set/'
# if not os.path.exists(output_dir):
#         os.makedirs(output_dir, exist_ok=True)
img_num = 0
while len(os.listdir(output_dir)) < 300:
    url = df.iloc[index]['url']
    
    response = requests.get(url)
    image = None
    if response.status_code == 200:
        image = Image.open(BytesIO(response.content))
    else:
        index += 1
        continue
    
    width, height = image.size
    
    # Keep only images >= 2k resolution
    if width < 3840 and height < 2160:
        index += 1
        continue

    # Cropping image to 2560x1440 by getting a box in the center of larger image
    # Center box on image
    target_width, target_height = 3840, 2160
    left = (width - target_width) // 2
    top = (height - target_height) // 2
    right = left + target_width
    bottom = top + target_height
    crop_box = (left, top, right, bottom)
    cropped_image = image.crop(crop_box)
    
    cropped_image.save(f'{output_dir}image_{img_num}.jpg')
    img_num += 1
    index += 1    